In [77]:
structure_file='data.vasp'
FC_file='FORCE_CONSTANTS_2ND'

In [78]:
from thermal_conductivity_AF import get_Vij, get_Sij
Vx, Vy, Vz=get_Vij(structure_file,FC_file)


In [79]:
import numpy as np 
eigenvector=np.loadtxt('eigr.dat').reshape((24,24))
omega=np.loadtxt('freq.dat')

In [80]:
Sx,Sy,Sz=get_Sij(Vx,Vy,Vz,eigenvector,omega)

In [81]:
#check
GULP_Vijz=np.loadtxt('Vijz.dat').reshape((24,24))
np.where(np.abs(Vz-GULP_Vijz) > 0.001)

(array([], dtype=int64), array([], dtype=int64))

In [82]:
GULP_Vijy=np.loadtxt('Vijy.dat').reshape((24,24))
np.where(np.abs(Vy-GULP_Vijy) > 0.001)

(array([], dtype=int64), array([], dtype=int64))

In [83]:
GULP_Vijx=np.loadtxt('Vijx.dat').reshape((24,24))
np.where(np.abs(Vx-GULP_Vijx) > 0.001)

(array([], dtype=int64), array([], dtype=int64))

In [84]:
GULP_Sijx=np.loadtxt('Sijx.dat').reshape((24,24))
GULP_Sijy=np.loadtxt('Sijy.dat').reshape((24,24))
GULP_Sijz=np.loadtxt('Sijz.dat').reshape((24,24))
np.where(np.abs(Sx-GULP_Sijx) > 0.01)

(array([ 2,  2,  2,  2,  2,  2,  9, 12, 14, 15, 21, 22]),
 array([ 9, 12, 14, 15, 21, 22,  2,  2,  2,  2,  2,  2]))

システムの違いでどうしてもズレが出る

In [85]:
np.sum(np.abs(Sx-GULP_Sijx) )

0.604556369400741

In [86]:
np.where(np.abs(Sy-GULP_Sijy) > 0.1)

(array([ 2, 21]), array([21,  2]))

In [87]:
Sy[2]

array([ 0.00000000e+00, -0.00000000e+00, -1.61922382e-17,  7.18477546e+01,
        1.74766510e+02, -1.65249442e+02, -4.42425771e+01,  8.13632681e+00,
        1.42493180e+02, -6.76131435e+01, -2.97489709e+01,  1.24603810e+01,
       -5.87304130e+01, -1.86498842e+01,  1.09191898e+02, -2.18974177e+01,
        2.31502851e+02,  2.17869646e+02, -2.01147569e+02,  2.21005046e+02,
       -3.28542407e+01, -3.40444808e+03, -1.61679958e+00,  9.42591723e+01])

In [88]:
GULP_Sijy[2]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  7.18349657e+01,
        1.74735402e+02, -1.65220028e+02, -4.42347020e+01,  8.13487854e+00,
        1.42467817e+02, -6.76011084e+01, -2.97436756e+01,  1.24581631e+01,
       -5.87199590e+01, -1.86465646e+01,  1.09172462e+02, -2.18935200e+01,
        2.31461643e+02,  2.17830865e+02, -2.01111765e+02,  2.20965708e+02,
       -3.28483926e+01, -3.40384209e+03, -1.61651179e+00,  9.42423942e+01])

In [89]:
np.sum(np.abs(Sz-GULP_Sijz) )

2.196118214102145

In [90]:
np.sum(np.abs(Sy-GULP_Sijy) )

1.9097769167402316

In [91]:
#mode diffusivity
#consider that frequency has cm-1 unit
#2pi of hbar is cancelled by this frequency definition

#Boltzmanns constant (J/K)
boltz = 1.38066244e-23

#Planck's constant (Js)
planck = 6.62617636e-34

#Speed of light (in cm/s)
speedl = 2.99792458e10

#eV -> Joules

evtoj = 1.6021917e-19

#Avogadros number
avogadro = 6.022045e23

In [92]:
fscale = np.sqrt(1.0e23*evtoj*avogadro)
fscale = fscale/(2.0*np.pi*speedl)

In [93]:
constant = ((1.0e-17*evtoj*avogadro)**0.5)*(fscale**3)
constant = np.pi*constant/48.0

In [94]:
constant

9.116410435342889

In [95]:
#get mean of frequency spacing
dwavg=0.0
for i in range(len(omega)-1):
    if omega[i] > 0.0:
        dwavg+=omega[i+1]-omega[i]
    elif omega[i+1] >0.0:
        dwavg+=omega[i+1]

dwavg=dwavg/(len(omega)-1)
dwavg

25.643677400657612

In [96]:
Di=np.zeros(len(omega))
nmodes=len(omega)
bfactor=5.0
broad = bfactor*dwavg
for i in range(nmodes):
    Di_loc = 0.0
    for j in range(nmodes):
        if(omega[i] > 0.01):
            dwij = (1.0/np.pi)*broad/( (omega[j] - omega[i])**2 + broad**2 )
            if(dwij > 0.01):
                Di_loc = Di_loc + dwij*Sx[j,i]**2+dwij*Sy[j,i]**2+dwij*Sz[j,i]**2
    Di[i] = Di[i] + Di_loc*constant/(omega[i]**2)

0.0006212297709835935
0.0006212298031355427
0.0006212298883337934
0.0024825603692520937
0.0024825603692520937
0.0024825603692520937
0.0024825603692520937
0.0024825603692520937
0.0024825603692520937
0.0006851229490440273
0.0006851229490440273
0.0006851229490440273
0.0006851229490440224
0.0006851229490440224
0.0006851229490440224
0.0003956370829438497
0.0003956370829438497
0.0003956370829438497
0.0003956370829438497
0.0003956370829438497
0.00039563708294384744
0.00026892498648857423
0.00026892498648857423
0.00026892498648857423
0.0006212297709835935
0.0006212298031355427
0.0006212298883337934
0.0024825603692520937
0.0024825603692520937
0.0024825603692520937
0.0024825603692520937
0.0024825603692520937
0.0024825603692520937
0.0006851229490440273
0.0006851229490440273
0.0006851229490440273
0.0006851229490440224
0.0006851229490440224
0.0006851229490440224
0.0003956370829438497
0.0003956370829438497
0.0003956370829438497
0.0003956370829438497
0.0003956370829438497
0.00039563708294384744
0.000

In [97]:
Di

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 7.02771176e-04,
       6.65331312e-03, 5.66888287e-03, 1.35436158e-03, 1.65159964e-04,
       5.80953543e-03, 2.17454654e-04, 1.95484201e-05, 5.54808794e-06,
       1.49635902e-04, 1.59060399e-05, 8.33568700e-04, 3.47256761e-05,
       6.16828619e-04, 5.28512417e-04, 2.96766631e-04, 4.86161109e-04,
       1.57728271e-05, 3.59178444e-02, 3.37691754e-02, 2.20085667e-03])

In [98]:
from ase.io import read
atoms=read(structure_file,format='vasp')
vol = atoms.get_volume()
kappafct = 1.0e30/vol
temperature=300
cmfact = planck*speedl/(boltz*temperature)
for i in range(nmodes):
    xfreq = omega[i]*cmfact
    expfreq = np.exp(xfreq)
    cv_i = boltz*xfreq*xfreq*expfreq/(expfreq - 1.0)**2
    print(omega[i],Di[i]*1.0e4,cv_i*kappafct*Di[i])

-1.365903968777618e-05 0.0 0.0
-5.998889625991092e-06 0.0 0.0
1.429945356769956e-05 0.0 0.0
221.940168139047 7.027711760173907 54.58689654572006
221.940168139047 66.53313115021618 516.7880059539896
221.940168139047 56.68882873571746 440.3235839908829
221.940168139047 13.543615833952998 105.19838912184892
221.940168139047 1.6515996406851294 12.828599378810692
221.940168139047 58.095354331179294 451.2486005237868
429.619338912949 2.1745465414892746 13.173987184542792
429.619338912949 0.1954842005637124 1.1842958078253862
429.619338912949 0.05548087935728142 0.3361180732142059
429.61933891295 1.4963590245454683 9.065344998013305
429.61933891295 0.159060398540048 0.9636306291700468
429.61933891295 8.335686996170974 50.49983137482455
516.419660702354 0.3472567611195146 1.8192920960998618
516.419660702354 6.168286187830099 32.31589867918807
516.419660702354 5.285124168515108 27.688977446220708
516.419660702354 2.967666313321079 15.547722815477956
516.419660702354 4.861611086905597 25.4701754